In [ ]:
# SncRNA 3’-ends
# snoRNAs

# https://www.ncbi.nlm.nih.gov/gene/167153
# https://github.com/yuma-m/synthesizer
# https://www.rcsb.org/structure/6LBJ
import numpy as np

In [2]:
import requests

protein_ID = '6LBJ'
url = 'https://files.rcsb.org/download/{}.pdb'.format(protein_ID.lower())
response = requests.get(url)
data = response.text
text = data.split("\n") #split data into array using newline as delimiter

In [3]:
# Dictionary to be used for converting amino acid abbreviations
aa_3to1 = {'CYS': 'C', 'ASP': 'D', 'SER': 'S',
'GLN': 'Q', 'LYS':'K','ILE': 'I', 'PRO': 'P',
 'THR': 'T', 'PHE': 'F', 'ASN': 'N', 'GLY': 'G',
 'HIS': 'H', 'LEU': 'L', 'ARG': 'R', 'TRP': 'W',
 'ALA': 'A', 'VAL':'V', 'GLU': 'E', 'TYR': 'Y', 'MET': 'M'}

In [4]:
import re
coordinates = []
protein_seq = []
for line in text:
    if re.match('ATOM',line):       # Matches lines beginning with 'ATOM'
        if(line[13:15] == 'CA'):    # Matches lines with c-alpha coordinates
            # Extract x, y, and z coordinates and append to list
            x = float(line[32:38])      
            y = float(line[40:46])     
            z = float(line[48:54])     
            coordinates.append([x,y,z]) 
            # Extract amino acid sequence and append to list
            aa = aa_3to1[line[17:20]]
            protein_seq.append(aa)

In [5]:
import pandas as pd
nu_df = pd.read_csv('aa_freq.csv', sep=',',header=0)
# replace NA's with 0's
nu_df.fillna(0,inplace=True)

In [6]:
from scipy.constants import speed_of_light as speedlight
#convert to np array
nu = nu_df.values
for ro in range(nu.shape[0]):
    for co in range(nu.shape[1]):
        if co == 0:
        #Convert amino acid code from 3 letter to 1 letter
            aa = nu[ro][co]
            aa = aa[0:3].upper()
            nu[ro][co]= aa_3to1[aa]
        else:
        # Convert from wavenumber to frequency
        # wavelength = 1/wavenumber
        # frequency = speed of light / wavelength
            if nu[ro][co] > 0:
                freq = speedlight / (1/nu[ro][co])

In [7]:
aa_list = nu[:,0]      # list of amino acids
nu = nu[:,1:]          # remove first column                
max_freq = nu.max()    # max and min 
min_freq = nu.min()    # need to exclude first column
human_range = 20000 - 20
for ro in range(nu.shape[0]):
    for co in range(nu.shape[1]):
        if nu[ro][co] > 0:
            # First scale to 0 to 1
            nu[ro][co] = (nu[ro][co] - min_freq)/ max_freq - min_freq
            # Scale to human hearing range 20 - 20000 Hz
            nu[ro][co] = (nu[ro][co] * human_range) + 20

In [8]:
import numpy as np

def aa_note(aa,length):
    row = np.where(aa_list == aa)[0][0]
    wave = np.zeros(int(44100 * float(length)))
    harm = 0
 
    for freq in nu[row]:
        if freq == 0:
            break
        else:
            volume = np.exp(-harm)
            phases = np.cumsum(2.0 * np.pi * freq / 44100 * np.ones(int(44100 * float(length))))
            wave += np.sin(phases) * volume
            harm += 1
    # scale wave to values between 1 and -1
    wave = 2.*(wave - wave.min())/np.ptp(wave)-1
    return wave

In [9]:
from synthesizer import Player, Synthesizer
player = Player()
player.open_stream()

ALSA lib pcm_dsnoop.c:567:(snd_pcm_dsnoop_open) unable to open slave
ALSA lib pcm_dmix.c:1000:(snd_pcm_dmix_open) unable to open slave
ALSA lib pcm.c:2721:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2721:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2721:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib pcm_dmix.c:1000:(snd_pcm_dmix_open) unable to open slave


In [10]:
# a sample protein sequence
protein_sequence = ''.join(protein_seq)
unit_length = 0.2
melodic_sequence = []
for residue in protein_sequence:
    sound_wave = aa_note(residue,unit_length)
    melodic_sequence = melodic_sequence + sound_wave.tolist()
melody = np.asarray(melodic_sequence)
melody = 2*(melody - melody.min())/np.ptp(melody)-1

In [11]:
player.play_wave(melody)


In [12]:
from scipy.io.wavfile import write

# Save the melody to a WAV file
write("melody.wav", 44100, melody.astype(np.float32))

In [12]:
import numpy as np
from scipy.spatial import Delaunay
# Convert to numpy array and tessellate
coordinates = np.asarray(coordinates,dtype=float)
tess = Delaunay(coordinates)

neighbors = {}
# Go through each simplex, or groups of four neighbors
for simplex in tess.simplices:  
    for vertex in simplex:
        # if neighbor group exists, add all neighbors and keep unique ones
        if vertex in neighbors.keys():
            neighbors[vertex] = np.unique(np.append(neighbors[vertex],simplex))
        # if neighbor group doesn't exist, then create one
        else:
            neighbors[vertex] = simplex

In [13]:
# dictionary that maps each amino acid to a row 
aa2row = {}
aa = 'CDSQKIPTFNGHLRWAVEYM'
for i,a in enumerate(aa):
    aa2row[a] = i


In [39]:
# get the protein sequence from the pdb file

In [14]:
# adj matrix

# create empty matrix
music_matrix = np.zeros((20,len(protein_seq)),dtype='int') 
# go through each position in the protein sequence
for i in range(len(protein_seq)):
    # go through the neighbors at each position
    for neighbor in neighbors[i]:
        # exclude melody from accompaniment
        if neighbor == i: 
            pass
        # change cell value at row (amino acid) and col (position) 
        # from 0 to 1
        else:
            row = aa2row[protein_seq[neighbor]]
            col = i
            music_matrix[row][col] = 1

In [15]:
for row in range(20):
    for col in range(len(protein_seq)-1,0,-1):
        if music_matrix[row][col-1] == 1:
            music_matrix[row][col-1] += music_matrix[row][col]
            music_matrix[row][col] = 0

In [16]:
# apply envelope to reduce clipping
secondary_structs = []
with open("protein_surface.dssp", "r") as f:
    secondary_struct_text = f.readlines()
for line in secondary_struct_text[28:]:
    if line[13] != "!":
        secondary_structs.append(line[16:18].strip())

secondary_structure = np.array(secondary_structs)

In [17]:
# go through each row in music matrix (amino acid)
notes = []
for row in range(20):
    # create empty note
    note = []
    # begin at column 0
    col = 0
    while(col < len(protein_seq)):
        # check secondary structure at that position to
        # determine note length
        if secondary_structure[col] == 'E':
            secondary_structure_factor = 2
        elif secondary_structure[col] == 'H':
            secondary_structure_factor = 0.5
        else:
            secondary_structure_factor = 1
        # build 'track' for each residue
        res = aa[row] # which amino acid note to use
        notelength = music_matrix[row][col] # length of the note
        if notelength == 0:
            # if length is zero then create note of correct length with volume 0
            wave = np.zeros(int(44100 * float(unit_length * secondary_structure_factor)))
            col += 1
        else:
            # if length is not zero, modify by secondary structure value
            # at each position over length of note
            total_length = 0
            for i in range(notelength):
                if secondary_structure[col+i] == 'E':
                    total_length += unit_length*2
                elif secondary_structure[col+i] == 'H':
                    total_length += unit_length*0.5
                else:
                    total_length += unit_length*1
            # build the note
            sound_wave = aa_note(res,total_length)
            # apply envelope to reduce clipping
            #wave = envelope(sound_wave,total_length)
            wave = sound_wave
            # move through matrix to next column with a 0
            col += int(notelength)
        # add new note to the track
        note = note + wave.tolist()
    # correct for differences in the track lengths due to rounding
    gap = len(notes) - len(note)
    filler = [0.0]*gap
    note = note + filler
    # superpose note on notes, this is creating harmonies
    print(len(note))
    notes.append(note)

5190565
5190562
5190559
5190562
5190561
5190564
5190567
5190559
5190564
5190566
5190565
5190566
5190565
5190562
5190569
5190560
5190567
5190567
5190569
5190570


In [18]:
obs_lengths = [len(a) for a in notes]
max_length = max(obs_lengths)

In [19]:
arr = np.zeros((len(notes),max_length))
for i in range(len(notes)):
    arr[i,:len(notes[i])] = notes[i]

In [20]:
harmony = 0.1 * arr.sum(axis = 0)

In [21]:
#include melody
melody = []
for i in range(len(protein_seq)):
    residue = protein_seq[i]
    if secondary_structure[i] == 'E':
        length = 2.0*unit_length 
    elif secondary_structure[i] == 'H':
        length = 0.5*unit_length 
    else:
        length = 1*unit_length    
    sound_wave = aa_note(residue,length)
    env_wave = sound_wave#envelope(sound_wave,length)
    melody = melody + env_wave.tolist()
melody = melody + [0.0]*9
melody = np.asarray(melody)
#rescale to a bit less thanrange [-1,1]
melody = 1.4*(melody - melody.min())/np.ptp(melody)-0.7

In [22]:
# fill in gaps
gap = len(melody) - len(harmony)
if gap > 0:
    filler = [0.0]*gap
    harmony = harmony.tolist() + filler

gap = len(harmony) - len(melody)
if gap > 0:
    filler = [0.0]*gap
    melody = melody + filler



In [26]:
from scipy.io.wavfile import write

write("melody_2.wav", 44100, melody.astype(np.float32))
write("harmony_2.wav", 44100, np.array(harmony).astype(np.float32))

In [28]:
player.play_wave(np.array(harmony))

In [29]:
complete_music = harmony + melody
#rescale to a bit less than range [-1,1]
complete_music = 1*(complete_music - complete_music.min())/np.ptp(complete_music)-0.5

In [34]:
player.play_wave(complete_music)

In [51]:
# save complete music to wav
write("complete_music.wav", 44100, complete_music.astype(np.float32))